In [1]:
#LIBRARIES
import pandas as pd
import re
import numpy as np

## nomalice parameters
from unidecode import unidecode

import spacy  # For preprocessing
from scipy import spatial
from time import time  # To time our operations
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

from tqdm import tqdm

from deep_translator import GoogleTranslator
nlp = spacy.load("en_core_web_sm")

## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
## nomalice parameters
from unidecode import unidecode

C:\Users\War\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
df = pd.read_excel("Profiles required for a representative sample of 600.xlsx")

In [4]:
#MINUSCULA

df["dict"] =  df["dict"].str.lower()
df["Bootcamps_unif"] =  df["Bootcamps_unif"].str.lower()

In [5]:
#FUNCIÓN PARA TRADUCIR

def translateString(to_translate, languageFinal):
    translated = GoogleTranslator(source='auto', target=languageFinal).translate(to_translate)
    return translated

#FUNCIÓN PARA LIMPIAR
nlp = spacy.load("en_core_web_sm")

def cleaning(text):
    ##text = text.lower()
    #text = text.replace('[>/,\.!?\-!?\n\)\(\r]', ' ') 
    text = re.sub("[^A-Za-z']+", ' ', str(text))
    doc = nlp(text)
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    #txt = [token.lemma_ for token in doc if not token.is_stop]
    txt = [tok.lemma_.lower() for tok in doc
                if not tok.is_stop and not tok.is_punct and tok.is_alpha ]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    ## return an array of ewords
    ##return txt
    
    rta = ""
    for t in txt:
        print(t)
        rta+= " "+t
    return rta

# HOMOGENIZACIÓN INICIAL (VOLVER A LA ESTRUCTURA INICIAL)

In [6]:

#FUNCIÓN QUE CAPTURA HETEROGENEIDADES QUE HUBO EN LA RECOLECCIÓN DE DATOS

def homo(q):
    
    q = q.replace("\n", " ") #SALTOS
    q = " ".join(q.split()) #ESPACIOS
    q = q.replace("<!---->", " ")

    H = re.findall("[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]", q) #20162020
    for i in H: 
        q = q.replace(i, f"'start': '{i[0:4]}', 'end': '{i[4:8]}'")
    
    K = re.findall('<span class="pv-entity__secondary-title separator">[^0-9]*</span>', q) # '<span class="pv-entity__secondary-title separator">Jornada completa</span>'
    for i in K:
        q = q.replace(i, "")
        
    Q = re.findall('[A-z][A-z][A-z] [0-9][0-9][0-9][0-9]', q) #SEP 2020
    for i in Q:
        q = q.replace(i, f"{i[4:8]}")

    educ = str(q.split("}],")[0]+"}]")
    job_title = str(q.split("}],'")[1].split("',[{'")[0])
    if "job_title" in q:
        job_history = str("[{'"+q.split("',[{'")[1])
    else:
        job_history = ""

    return educ, job_title, job_history



In [7]:
#LOOP PARA EDUCACIÓN

df["educ"] = np.nan;
df["dict"] = df["dict"].replace(np.nan, "")

for i in tqdm(range(len(df))):
    try:
        if len(df["dict"][i])>0:
            df["educ"][i] = homo(df["dict"][i])[0]
    except:
        None

  0%|          | 0/600 [00:00<?, ?it/s]<ipython-input-7-b46f5e288b41>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["educ"][i] = homo(df["dict"][i])[0]
C:\Users\War\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
100%|██████████| 600/600 [00:00<00:00, 2110.60it/s]


In [8]:
#LOOP PARA JOB TITLE (este en realidad es el encabezado, hubo gente que no tenía trabajo: puede no coincidir con el trabajo pero sirve para la gente que no tiene historial)

df["job_title"] = np.nan;
df["dict"] = df["dict"].replace(np.nan, "")

for i in tqdm(range(len(df))):
    try:
        if len(df["dict"][i])>0:
            df["job_title"][i] = homo(df["dict"][i])[1]
    except:
        None

  0%|          | 0/600 [00:00<?, ?it/s]<ipython-input-8-7f3ed8e1b60a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["job_title"][i] = homo(df["dict"][i])[1]
100%|██████████| 600/600 [00:00<00:00, 1978.79it/s]


In [9]:
#LOOP PARA HOB HISTORY

df["job_history"] = np.nan;
df["dict"] = df["dict"].replace(np.nan, "")

for i in tqdm(range(len(df))):
    try:
        if len(df["dict"][i])>0:
            df["job_history"][i] = homo(df["dict"][i])[2]
    except:
        None

  0%|          | 0/600 [00:00<?, ?it/s]<ipython-input-9-797f3fa8e5ba>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["job_history"][i] = homo(df["dict"][i])[2]
100%|██████████| 600/600 [00:00<00:00, 2158.87it/s]


# PARTIR DE LOS ALGORITMOS PREVIOS: OBTENER EDUCACION, HISTORIAL LABORAL Y TITULO ACTUAL HOMOGENIZADO Y TRADUCIDO

In [10]:
#HOMOGENIZACIÓN DE EDUCACION

L = []

for i in df["educ"]:
    l = str(i).split("},") #convertir en string la formación del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for formacion in l: #para cada formación
        formacion = formacion.replace("[", "").replace("]", "") + "}" #separelas y conviertalas en diccionarios
        k.append(formacion) #añadalas a la lista de formación
    L.append(k) #añada la lista de formación para individuo

#VER UN EJEMPLO: Esta es la última formación del primer individuo desordenada en diccionario pero como un string   
L[0][0]


K = [] #LISTA D ELISTAS EN DONDE SE ALMACENARAN LAS FORMACIONES A MANERA DE DICCIONARIO PARA CADA INDIVIDUO

for individuo in tqdm(L): #para cada individuo

    educacion_ind = [] #creo una lista de sus educaciones

    for ins in individuo: # para cada cada formación dentro de las formaciones de cada individuos
        ins = ins.replace('"', "'")

        c = {}

        #CADA CAMPO SE EXTRAE POR MEDIO DE RegEx, COMUNES A TODOS LAS FORMACIONES STR DE LOS INDIVIDUOS EN EL DATASET
        #CADA CAMPO SE EXTRAE EN CASO DE QUE EXISTA DE LO CONTRARIO NO

        #EXTRAER EDUCACIÓN
        if ( len(re.findall("'education_org': '.*?'", ins)) ) > 0:
            c["education"] =  re.findall("'education_org': '.*?'", ins)[0].replace("'education_org': ", "").replace("'", "")

        #EXTRAER DEGREE
        if ( len(re.findall("'degrees': '.*?'", ins)) ) > 0:
            c["degrees"] =  re.findall("'degrees': '.*?'", ins)[0].replace("'degrees': ", "").replace("'", "")

        #EXTRAER SUMMARY
        if ( len(re.findall("'summary': '.*?'", ins)) ) > 0:
            c["summary"] =  re.findall("'summary': '.*?'", ins)[0].replace("'summary': ", "").replace("'", "")


        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")
            c["start"] = (c["start"])

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")
            c["end"] = (c["end"])

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
K[0][0]

df["educ_history_clean"] = K


100%|██████████| 600/600 [00:00<00:00, 1994.91it/s]


In [204]:


for i in tqdm(K):
    for e in tqdm(i):
        if "degrees" in e and len(e["degrees"])>0:
           try:
               e["degrees"] = translateString(str(e["degrees"]), "en")
               e["degrees"] = cleaning(e["degrees"])
           except:
              None

        if "education" in tqdm(e) and len(e["education"])>0:
           try:
            e["education"] = translateString(str(e["education"]), "en")
            e["education"] = cleaning(e["education"])
           except:
              None

df["educ_history_clean"] = K

  0%|          | 0/4 [00:00<?, ?it/s]

nanotechnology
engineer
honor
degree


bolivarian
pontifical
university



  0%|          | 0/4 [00:00<?, ?it/s]

control
engineering
year
degree


...

tea



  0%|          | 0/4 [00:00<?, ?it/s]

lic
labor
relation




  0%|          | 0/3 [00:00<?, ?it/s]

national
slaughter
university


In [11]:
#HOMOGENIZAR HISTORIAL

#SEPARANDO LOS STRINGS DE CADA TRABAJO

L = []

for i in df.job_history:
    l = str(i).split("},") #convertir en string el job history del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for job in l: #para cada trabajo
        job = job.replace("[", "").replace("]", "") + "}" #separelos y conviertalas en diccionarios
        k.append(job) #añadalas a la lista de jobs
    L.append(k) 

#EXTRAER DE MANERA ORDENADA EL JOB HISTORY, LA FECHA DE TERMINACIÓN Y COMIENZO

K = [] 

for individuo in tqdm(L): 

    educacion_ind = [] 

    for ins in individuo: 
        ins = ins.replace('"', "'")

        c = {}

        #EXTRAER JOB-TITLE
        if ( len(re.findall("'job_title': '.*?'", ins)) ) > 0:
            c["job_title"] =  re.findall("'job_title': '.*?'", ins)[0].replace("'job_title': ", "").replace("'", "")
            c["job_title"] = str(c["job_title"])
            # c["job_title"] = cleaning(c["job_title"])

        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")
            c["start"] = str((c["start"][::-1][0:4][::-1])) #SOLO CONSERVAR EL AÑO 

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")
            c["end"] = str((c["end"][::-1][0:4][::-1])) #SOLO CONSERVAR EL AÑO

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general


df["job_history_clean"] = K



100%|██████████| 600/600 [00:00<00:00, 6225.51it/s]


In [ ]:
#TRADUCIR Y LIMPIAR 

for i in tqdm(K[:]):
    for e in tqdm(i):
        # A = "job_title" in e
        # if len(e["job_title"])>0 and A == True:
        try:
            e["job_title"] = translateString(str(e["job_title"]), "en")
            e["job_title"] = cleaning(e["job_title"])
        except:
            None


df["job_history_clean"] = K

100%|██████████| 600/600 [00:02<00:00, 222.27it/s]


In [12]:
#HOMOGENIZAR TITULO ACTUAL: TRADUCIR

for i in tqdm(range(len(df))):
    
    df["job_title"][i] = translateString(str(e["job_title"][i]), "en")
    df["job_title"][i] = cleaning(df["job_title"][i])
    

  0%|          | 0/600 [00:00<?, ?it/s]


NameError: name 'e' is not defined

# ENCONTRAR EL AÑO DE GRADUACIÓN

In [13]:
# ENCONTRAR MATCH

df["bootcamp_educ"] = np.nan
df["ratio"] = np.nan

for i in range(len(df)):
    for h in range(len(df.educ_history_clean[i])):
        if "education" in df.educ_history_clean[i][h]:

            wordLen = len(df.educ_history_clean[i][h]["education"])
            ratio = fuzz.partial_ratio(df.Bootcamps_unif[i], df.educ_history_clean[i][h]["education"])
            if ratio>=85:
                df["bootcamp_educ"][i] = df.educ_history_clean[i][h]["education"]
                df["ratio"][i] = ratio

            if  df.educ_history_clean[i][h]["education"] == "hola&lt;​code/&gt;":
                df["bootcamp_educ"][i] = df.educ_history_clean[i][h]["education"]
                df["ratio"][i] = ratio

dfk = df[df["bootcamp_educ"].notna()] #40 quedan sin hacer match


<ipython-input-13-1b1af8cf3323>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bootcamp_educ"][i] = df.educ_history_clean[i][h]["education"]
<ipython-input-13-1b1af8cf3323>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ratio"][i] = ratio
<ipython-input-13-1b1af8cf3323>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bootcamp_educ"][i] = df.educ_history_clean[i][h]["education"]
<ipython-input-13-1b1af8cf3323>:18: Se

In [14]:
dfk["bootcamp_educ"].unique()

array(['digital house', 'digital house argentina', 'digital house ',
       'digital house brasil', 'digital house brazil', 'acámica',
       'acamica', 'acámica (globant - ibm)', 'laboratoria', 'ironhack',
       'holberton school', 'desafío latam', 'ada itw', 'ada',
       'hola&lt;\u200bcode/&gt;', 'hola', 'le wagon',
       'plataforma 5 - coding bootcamp', 'hack academy', 'hack academy ',
       'mindhub la', '4geeks academy', 'bogotá dev', 'muktek academy'],
      dtype=object)

In [15]:
dfk = dfk.reset_index(drop = True)

dfk["end_boot"] = np.nan 
dfk["end_boot_cat"] = np.nan

c = 0

for i in range(len(dfk.educ_history_clean)):
    for e in range(len(dfk.educ_history_clean[i])): #DENTRO DE LA EDUCACIÓN DE CADA INDIVIDUO
        if "education" in dfk.educ_history_clean[i][e].keys() and dfk.educ_history_clean[i][e]["education"] == dfk.bootcamp_educ[i]:

            if "end" in dfk.educ_history_clean[i][e].keys():
                dfk["end_boot"][i] = dfk.educ_history_clean[i][e]["end"]
                dfk["end_boot_cat"][i] = "end"
                
            elif "start" in dfk.educ_history_clean[i][e].keys():
                dfk["end_boot"][i] = dfk.educ_history_clean[i][e]["start"] #SI NO TENÍA EL FINAL POR LO MENOS SE LE IMPUTÓ EL INICIAL, EXISTE LA POSIBILIDAD DE QUE NO LO HAYA TERMINADO
                dfk["end_boot_cat"][i] = "start"



for i in range(len(dfk.end_boot)):
    if dfk.end_boot[i] == "":
        dfk.end_boot[i] = 2020
        dfk.end_boot_cat[i] = "inp"


dfk["end_boot"][0]

<ipython-input-15-835df4ac2b73>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfk["end_boot"][i] = dfk.educ_history_clean[i][e]["end"]
<ipython-input-15-835df4ac2b73>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfk["end_boot_cat"][i] = "end"
<ipython-input-15-835df4ac2b73>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfk.end_boot[i] = 2020
<ipython-input-15-835df4ac2b73>:25: SettingWithCopyWarning: 
A value is trying to 

2023.0

In [17]:
dfk.to_excel("data_base_VF.xlsx", index = False)

# ENCONTRAR EL TITULO